# Fedbiomed Researcher base example with OPACUS

Use for developing (autoreloads changes made across packages)

In this notebook we show how `opacus` (https://opacus.ai/) can be used in Fed-BioMed. Opacus is a library which allows to train PyTorch models with differential privacy. We will train the basic MNIST example using two nodes.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)

2022-02-11 16:25:02,959 fedbiomed INFO - Component environment:
2022-02-11 16:25:02,960 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-02-11 16:25:04,282 fedbiomed INFO - Messaging researcher_5b91b9f1-8e80-4df9-9c2b-5f07d56a2127 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x1113259d0>
2022-02-11 16:25:04,320 fedbiomed INFO - Listing available datasets in all nodes... 
2022-02-11 16:25:04,367 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / DEBUG - Message received: {'researcher_id': 'researcher_5b91b9f1-8e80-4df9-9c2b-5f07d56a2127', 'command': 'list'}
2022-02-11 16:25:04,373 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / DEBUG - Message received: {'researcher_id': 'researcher_5b91b9f1-8e80-4df9-9c2b-5f07d56a2127', 'command': 'list'}
2022-02-11 16:25:14,330 fedbiomed INFO - 
 Node: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc | Number of Datasets: 1 
+--------+-------------+-----

{'node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': 'MNIST database',
   'shape': [60000, 1, 28, 28]}],
 'node_11bdb502-9832-408f-b421-14890fc21948': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': 'MNIST database',
   'shape': [60000, 1, 28, 28]}]}

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default)
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Define an experiment model and parameters

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [3]:
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+'/')
model_file = tmp_dir_model.name + '/class_export_mnist.py'

In the cell below, we are going to define the model using opacus for differential privacy. For this example, we are going to use the function `make_private` from `opacus.privacy_engine`. Two hyperparameters should be defined:
* `noise_multiplier`: The ratio of the standard deviation of the Gaussian noise to the L2-sensitivity of the function to which the noise is added (How much noise to add)
* `max_grad_norm`: The maximum norm of the per-sample gradients. Any gradient with norm higher than this will be clipped to this value.

It is worth noting that in order to use the opacus `PrivacyEngine` class we need to properly define as training plan attributes a `model`, a `dataloader` and an `optimizer`.

In [4]:
%%writefile "$model_file"

import torch
import torch.nn as nn
import torch.nn.functional as F
from fedbiomed.common.torchnn_refactor import TorchTrainingPlan
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from opacus import PrivacyEngine 

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args):
        super(MyTrainingPlan, self).__init__()
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
                "import torch.nn.functional as F",
                "from torch.utils.data import DataLoader",
                "from opacus import PrivacyEngine",]
        self.add_dependency(deps)
        
        self.model = self.make_model()
        
        self.noise_multiplier = model_args['noise_multiplier']
        self.max_grad_norm = model_args['max_grad_norm']
        
    def make_model(self):
        model = nn.Sequential(nn.Conv2d(1, 32, 3, 1),
                                  nn.ReLU(),
                                  nn.Conv2d(32, 64, 3, 1),
                                  nn.ReLU(),
                                  nn.MaxPool2d(2),
                                  nn.Dropout(0.25),
                                  nn.Flatten(),
                                  nn.Linear(9216, 128),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(128, 10),
                                  nn.LogSoftmax(dim=1))
        return model
        
    def forward(self, x):
        return self.model(x)

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        
        # enter PrivacyEngine
        privacy_engine = PrivacyEngine()
        self.model, self.optimizer, data_loader = privacy_engine.make_private(module=self.model,
                                                                    optimizer=self.optimizer,
                                                                    data_loader=data_loader,
                                                                    noise_multiplier=self.noise_multiplier,
                                                                    max_grad_norm=self.max_grad_norm,
                                                                    )
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss
        
    def postprocess(self, params):
        # params keys are changed by the privacy engine (as _module.param_key): should be re-named
        params_keys = list(params.keys())
        for key in params_keys:
            if '_module' in key:
                newkey = key.replace('_module.', '')
                params[newkey] = params.pop(key)
        return params

Writing /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/tmpagii12j8/class_export_mnist.py


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side. For instance, the privacy parameters should be passed here.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [5]:
model_args = {'noise_multiplier':1., 'max_grad_norm':1.0}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 3, 
    'dry_run': False,  
    'batch_maxnum': 250 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `node_selection_strategy` between the rounds

In [6]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 3

exp = Experiment(tags=tags,
                 #nodes=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-02-11 16:25:25,847 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-02-11 16:25:25,860 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / DEBUG - Message received: {'researcher_id': 'researcher_5b91b9f1-8e80-4df9-9c2b-5f07d56a2127', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-02-11 16:25:25,863 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / DEBUG - Message received: {'researcher_id': 'researcher_5b91b9f1-8e80-4df9-9c2b-5f07d56a2127', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-02-11 16:25:35,856 fedbiomed INFO - Node selected for training -> node_11bdb502-9832-408f-b421-14890fc21948
2022-02-11 16:25:35,858 fedbiomed INFO - Node selected for training -> node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc
2022-02-11 16:25:35,861 fedbiomed INFO - Checking data quality of federated datasets...
2022-02-11 16:25:36,536 fedbiomed DEBUG - torchnn saved model filename: /Users/balelli/o

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the nodes

In [7]:
exp.run()

2022-02-11 16:25:41,916 fedbiomed INFO - Sampled nodes in round 0 ['node_11bdb502-9832-408f-b421-14890fc21948', 'node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc']
02/11/2022 16:25:41:INFO:Sampled nodes in round 0 ['node_11bdb502-9832-408f-b421-14890fc21948', 'node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc']
2022-02-11 16:25:41,920 fedbiomed INFO - Send message to node node_11bdb502-9832-408f-b421-14890fc21948 - {'researcher_id': 'researcher_5b91b9f1-8e80-4df9-9c2b-5f07d56a2127', 'job_id': 'a801321f-dfd5-490a-bd4d-5969448a5e6f', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {'noise_multiplier': 1.0, 'max_grad_norm': 1.0}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/02/11/my_model_ecfd902b-24cb-4ed0-945d-fdca01713c08.py', 'params_url': 'http://localhost:8844/media/uploads/2022/02/11/aggregated_params_init_965842f5-60af-49fe-a6ae-7960464167d8.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'n

2022-02-11 16:25:42,039 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_5b91b9f1-8e80-4df9-9c2b-5f07d56a2127', 'job_id': 'a801321f-dfd5-490a-bd4d-5969448a5e6f', 'params_url': 'http://localhost:8844/media/uploads/2022/02/11/aggregated_params_init_965842f5-60af-49fe-a6ae-7960464167d8.pt', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'training_data': {'node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc': ['dataset_aa6ea889-3743-48c9-8b3e-09f7aa7f3196']}, 'model_args': {'noise_multiplier': 1.0, 'max_grad_norm': 1.0}, 'model_url': 'http://localhost:8844/media/uploads/2022/02/11/my_model_ecfd902b-24cb-4ed0-945d-fdca01713c08.py', 'model_class': 'MyTrainingPlan', 'command': 'train'}
02/11/2022 16:25:42:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_5b91b9f1-8e80-4df9-9c2b-5f07d56a2127', 'job_id': 'a8

2022-02-11 16:26:58,673 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 1 [4060/60000 (6%)]	Loss: 2.152004
02/11/2022 16:26:58:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 1 [4060/60000 (6%)]	Loss: 2.152004
2022-02-11 16:27:00,838 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 1 [3850/60000 (6%)]	Loss: 2.126896
02/11/2022 16:27:00:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 1 [3850/60000 (6%)]	Loss: 2.126896
2022-02-11 16:27:06,039 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 1 [3520/60000 (6%)]	Loss: 2.168883
02/11/2022 16:27:06:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 1 [3520/60000 (6%)]	Loss: 2.168883
2022-02-11 16:27:08,466 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 1 [3680/60000 (6%)]	Loss: 2.08681

2022-02-11 16:28:56,118 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 1 [9450/60000 (17%)]	Loss: 1.690872
02/11/2022 16:28:56:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 1 [9450/60000 (17%)]	Loss: 1.690872
2022-02-11 16:28:59,647 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 1 [9460/60000 (18%)]	Loss: 1.501843
02/11/2022 16:28:59:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 1 [9460/60000 (18%)]	Loss: 1.501843
2022-02-11 16:29:03,123 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 1 [8800/60000 (18%)]	Loss: 1.408368
02/11/2022 16:29:03:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 1 [8800/60000 (18%)]	Loss: 1.408368
2022-02-11 16:29:05,847 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 1 [10810/60000 (18%)]	Loss:

2022-02-11 16:30:28,646 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 2 [4800/60000 (8%)]	Loss: 1.588488
02/11/2022 16:30:28:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 2 [4800/60000 (8%)]	Loss: 1.588488
2022-02-11 16:30:30,411 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 2 [5700/60000 (8%)]	Loss: 1.425355
02/11/2022 16:30:30:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 2 [5700/60000 (8%)]	Loss: 1.425355
2022-02-11 16:30:34,211 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 2 [4840/60000 (9%)]	Loss: 1.333084
02/11/2022 16:30:34:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 2 [4840/60000 (9%)]	Loss: 1.333084
2022-02-11 16:30:36,401 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 2 [5940/60000 (9%)]	Loss: 1.47578

...

02/11/2022 16:46:06:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 1 [8550/60000 (12%)]	Loss: 0.918622
2022-02-11 16:46:11,251 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 1 [8160/60000 (13%)]	Loss: 0.909449
02/11/2022 16:46:11:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 1 [8160/60000 (13%)]	Loss: 0.909449
2022-02-11 16:46:12,936 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 1 [8800/60000 (13%)]	Loss: 1.269679
02/11/2022 16:46:12:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 1 [8800/60000 (13%)]	Loss: 1.269679
2022-02-11 16:46:17,325 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 1 [8160/60000 (14%)]	Loss: 1.163921
02/11/2022 16:46:17:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 1 [8160/60000 (14%)]	Loss: 1.163921
2022-02

02/11/2022 16:47:35:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 2 [1600/60000 (3%)]	Loss: 1.144724
2022-02-11 16:47:39,439 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 2 [1960/60000 (3%)]	Loss: 1.314559
02/11/2022 16:47:39:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 2 [1960/60000 (3%)]	Loss: 1.314559
2022-02-11 16:47:41,944 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 2 [2600/60000 (4%)]	Loss: 1.985688
02/11/2022 16:47:41:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 2 [2600/60000 (4%)]	Loss: 1.985688
2022-02-11 16:47:45,299 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 2 [2400/60000 (4%)]	Loss: 1.200577
02/11/2022 16:47:45:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 2 [2400/60000 (4%)]	Loss: 1.200577
2022-02-11 16:

02/11/2022 16:49:29:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 2 [8100/60000 (14%)]	Loss: 1.460293
2022-02-11 16:49:31,885 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 2 [9880/60000 (15%)]	Loss: 1.613599
02/11/2022 16:49:31:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 2 [9880/60000 (15%)]	Loss: 1.613599
2022-02-11 16:49:38,471 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 2 [9690/60000 (15%)]	Loss: 1.961706
02/11/2022 16:49:38:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 2 [9690/60000 (15%)]	Loss: 1.961706
2022-02-11 16:49:40,441 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 2 [9200/60000 (16%)]	Loss: 2.436345
02/11/2022 16:49:40:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 2 [9200/60000 (16%)]	Loss: 2.436345
2022-02

02/11/2022 16:51:08:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 3 [3430/60000 (6%)]	Loss: 1.563822
2022-02-11 16:51:15,316 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 3 [4550/60000 (6%)]	Loss: 1.040126
02/11/2022 16:51:15:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 3 [4550/60000 (6%)]	Loss: 1.040126
2022-02-11 16:51:16,550 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 3 [3680/60000 (6%)]	Loss: 1.212851
02/11/2022 16:51:16:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 3 [3680/60000 (6%)]	Loss: 1.212851
2022-02-11 16:51:27,635 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 3 [3920/60000 (6%)]	Loss: 1.310409
02/11/2022 16:51:27:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 3 [3920/60000 (6%)]	Loss: 1.310409
2022-02-11 16:

02/11/2022 16:52:57:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 3 [9460/60000 (18%)]	Loss: 1.498662
2022-02-11 16:52:58,752 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 3 [9240/60000 (17%)]	Loss: 1.306097
02/11/2022 16:52:58:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 3 [9240/60000 (17%)]	Loss: 1.306097
2022-02-11 16:53:04,385 fedbiomed INFO - log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 3 [13800/60000 (18%)]	Loss: 1.174049
02/11/2022 16:53:04:INFO:log from: node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc / INFO - Train Epoch: 3 [13800/60000 (18%)]	Loss: 1.174049
2022-02-11 16:53:05,392 fedbiomed INFO - log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 3 [9240/60000 (18%)]	Loss: 1.524691
02/11/2022 16:53:05:INFO:log from: node_11bdb502-9832-408f-b421-14890fc21948 / INFO - Train Epoch: 3 [9240/60000 (18%)]	Loss: 1.524691
2022-

Local training results for each round and each node are available in `exp.training_replies` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [8]:
print("\nList the training rounds : ", exp.training_replies.keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies[rounds - 1].data
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies[rounds - 1].dataframe


List the training rounds :  dict_keys([0, 1, 2])

List the nodes for the last training round and their timings : 
	- node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc :    
		rtime_training=550.56 seconds    
		ptime_training=663.72 seconds    
		rtime_total=560.53 seconds
	- node_11bdb502-9832-408f-b421-14890fc21948 :    
		rtime_training=558.74 seconds    
		ptime_training=672.83 seconds    
		rtime_total=572.40 seconds




,success,msg,dataset_id,node_id,params_path,params,timing
0,True,,dataset_aa6ea889-3743-48c9-8b3e-09f7aa7f3196,node_5b8275b7-fc9b-4f8e-bf50-3f590a36effc,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'model._module.0.weight': [[tensor([[ 0.3336,...","{'rtime_training': 550.5559841049999, 'ptime_t..."
1,True,,dataset_03fb8252-1c9b-4d9d-9ec0-b6239c5c2df0,node_11bdb502-9832-408f-b421-14890fc21948,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'model._module.0.weight': [[tensor([[ 0.3907,...","{'rtime_training': 558.7415895699999, 'ptime_t..."


Federated parameters for each round are available in `exp.aggregated_params` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [9]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())


List the training rounds :  dict_keys([0, 1, 2])

Access the federated params for the last training round :
	- params_path:  /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0000/aggregated_params_b859995c-e4b6-4561-a8b7-8f020c2906b0.pt
	- parameter data:  odict_keys(['model.0.weight', 'model.0.bias', 'model.2.weight', 'model.2.bias', 'model.7.weight', 'model.7.bias', 'model.10.weight', 'model.10.bias'])


# Testing

We define a little testing routine to extract the accuracy metrics on the testing dataset

In [10]:
import torch
import torch.nn.functional as F


def testing_Accuracy(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    device = 'cpu'

    correct = 0
    
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

        pred = output.argmax(dim=1, keepdim=True)

    test_loss /= len(data_loader.dataset)
    accuracy = 100* correct/len(data_loader.dataset)

    return(test_loss, accuracy)

In [11]:
from torchvision import datasets, transforms
import os

local_mnist = os.path.join(environ['TMP_DIR'], 'local_mnist')

transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

test_set = datasets.MNIST(root = local_mnist, download = True, train = False, transform = transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/train-images-idx3-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw

Processing...


/Users/balelli/miniconda3/envs/fedbiomed-researcher/lib/python3.9/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [12]:
fed_model = exp.model_instance
fed_model.load_state_dict(exp.aggregated_params[rounds - 1]['params'])

acc_federated = testing_Accuracy(fed_model, test_loader)

print('\nAccuracy federated training:  {:.4f}'.format(acc_federated[1]))

print('\nError federated training:  {:.4f}'.format(acc_federated[0]))


Accuracy federated training:  86.3900

Error federated training:  0.5637
